## 0: Import the data

In [1]:
import os
os.getcwd()
os.chdir('C:\\Users\\zain.syed\\Desktop\\University of toronto\\3250 - Intro to Data Science\\project\\data\\cardataset')

In [2]:
import pandas as pd
import numpy as np

data= pd.read_csv('data.csv')



In [3]:
data.head(5)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


## 1.0: Pre-processing
1.1 Remove and/or Pre-process N/A variables

In [4]:
## Provides a summary of where the N/A values lie
## Categorical values are 'treated' on an individual basis
## whereas Numerical values are 'treated' in the pipeline
data.isnull().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3742
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

In [5]:
## Fills the N/A values in Market Category to other
data['Market Category']=data['Market Category'].fillna('Other')
data['Engine Fuel Type']= data['Engine Fuel Type'].fillna('Other')

In [6]:
## Provides a visual check of pre-processing the variables
data.isnull().sum()

Make                  0
Model                 0
Year                  0
Engine Fuel Type      0
Engine HP            69
Engine Cylinders     30
Transmission Type     0
Driven_Wheels         0
Number of Doors       6
Market Category       0
Vehicle Size          0
Vehicle Style         0
highway MPG           0
city mpg              0
Popularity            0
MSRP                  0
dtype: int64

1.2 Apply MultiLabelBinarizer to categorical variables

In [7]:
categorical_variables= ['Make','Model','Engine Fuel Type','Transmission Type','Vehicle Size',
                       'Vehicle Style']
numeric_variables= ['Year','Engine HP','Number of Doors','highway MPG',
                    'city mpg','Popularity','Engine Cylinders']

1.3 Separate MSRP into labels

In [8]:
MSRP= data['MSRP']
data= data.drop('MSRP', axis=1)


#### Setup

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

#### Numeric Pipeline

In [10]:
## 1. Imputer replaces missing values with the median value
## 2. Standard scaler scales the values between 0 and 1
numeric_pipeline= Pipeline([
    ('selector',DataFrameSelector(numeric_variables)),
    ('imputer',Imputer(strategy='median')),
    ('std_scaler', StandardScaler()),
    
])

numeric_data= numeric_pipeline.fit_transform(data)
numeric_data.shape

C:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


(11914, 7)

#### Categorical Pipeline

Reference:
1. https://stackoverflow.com/questions/46162855/fit-transform-takes-2-positional-arguments-but-3-were-given-with-labelbinarize

In [11]:


from sklearn.base import TransformerMixin #gives fit_transform method for free
from sklearn.preprocessing import MultiLabelBinarizer

class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = MultiLabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

categorical_pipeline= Pipeline([
    ('selector',DataFrameSelector(categorical_variables)),
    ('label_binarizer', MyLabelBinarizer()),
])


categoric_data= categorical_pipeline.fit_transform(data[categorical_variables])

categoric_data.shape



(11914, 995)

In [12]:
full_pipeline= FeatureUnion(transformer_list=
                            [
                           ('num_pipeline', numeric_pipeline),
                           ('cat_pipeline', categorical_pipeline),
                            ])


data_ml_prepared= full_pipeline.fit_transform(data)
data_ml_prepared.shape

(11914, 1002)

1.4 Split into training and test set

In [46]:
from sklearn.model_selection import train_test_split

train_data, test_data, MSRP_train,MSRP_test= train_test_split(np.array(data_ml_prepared),np.array(MSRP),
                                                              test_size=0.2,random_state=42)

In [47]:
print(train_data.shape,test_data.shape)

(9531, 1002) (2383, 1002)


In [48]:
print(MSRP_train.shape, MSRP_test.shape)

(9531,) (2383,)


array([29695, 30495, 37650, ..., 31750, 20995, 57700], dtype=int64)

##  2.0 Linear regression



2.1 Train the model

In [49]:
from sklearn.linear_model import LinearRegression

lin_reg= LinearRegression()

lin_reg.fit(train_data,MSRP_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

2.2 Test the model

In [50]:
from sklearn.metrics import mean_squared_error

car_msrp_predictions= lin_reg.predict(test_data)

from sklearn.metrics import mean_squared_error

lin_mse= mean_squared_error(MSRP_test,car_msrp_predictions)
lin_rmse= np.sqrt(lin_mse)

lin_rmse

311615236520228.44

2.3 Fix issues:
The first step was to look at the numbers using the head function, 
they seemed quite normal
The Rmse seems too big of a number, dtype is investigated

In [66]:
print(MSRP_test.dtype,car_msrp_predictions.dtype)

int64 float64


In [51]:
MSRP_test

array([29695, 30495, 37650, ..., 31750, 20995, 57700], dtype=int64)

In [71]:
import matplotlib.pyplot as plt

lin_mse= mean_squared_error(MSRP_test.astype('int'),car_msrp_predictions.astype('int'))
lin_rmse= np.sqrt(lin_mse)

lin_rmse

5789.53082881443

Our linear regression algorithm has an accuracy of ~ $ 5,800. 
In order to improve it, we can:

1. Use a correlation matrix and drop some variables
2. Try out a different algorithm

2.4 Try Ordinary least squares 

Reference:
1. https://stackoverflow.com/questions/38573354/multiple-linear-regression-in-python-patsyerror-model-is-missing-required-outc
2. http://www.statsmodels.org/stable/index.html

In [86]:


import statsmodels.formula.api as smf
df = pd.DataFrame({'x':train_data, 'y':MSRP_train})
model = smf.ols('y~x', data=df).fit()


Exception: Data must be 1-dimensional

In [83]:
train_data.ravel

<function ndarray.ravel>

2.3 Build an API to apply pipeline to extract info